In [97]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.metrics import accuracy_score
import os
# os.chdir('/Users/headout1/Documents/My Projects/similar')
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# Figures inline and set visualization style
%matplotlib inline
sns.set()

Version:  2.9.1
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [98]:
from sklearn.neighbors import NearestNeighbors
rec_model = NearestNeighbors(metric = 'cosine')

In [99]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [100]:
# Import dataset
df = pd.read_csv('/Users/headout1/Documents/My Projects/similar/experience.csv')
df.head()

,experience_id,experience_name,city,combined_id,sub_category_id,status
0,14281.0,Super Saver Tour - Auschwitz-Birkenau & Wielic...,Krakow,2197.0,1010.0,available
1,8128.0,Palmitos Park,Gran Canaria,3095.0,NaN,available
2,14461.0,Tickets To Thyssen-Bornemisza Museum with Trad...,Madrid,2930.0,1002.0,available
3,15328.0,Exterior Grand Walking Tour of Prague & Prague...,Prague,1214.0,1009.0,available
4,14253.0,Conservation Drive to Al Maha Desert Resort wi...,Dubai,262.0,1073.0,available


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2790 entries, 0 to 2789
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   experience_id    2789 non-null   float64
 1   experience_name  2789 non-null   object 
 2   city             2789 non-null   object 
 3   combined_id      1599 non-null   float64
 4   sub_category_id  2662 non-null   float64
 5   status           2790 non-null   object 
dtypes: float64(3), object(3)
memory usage: 130.9+ KB


In [102]:
df.isnull().sum()

experience_id         1
experience_name       1
city                  1
combined_id        1191
sub_category_id     128
status                0
dtype: int64

In [103]:
df = df.dropna(subset=['sub_category_id'])

In [104]:
df['combined_id']=df['combined_id'].fillna('exp -'+df['experience_id'].astype(str))

In [105]:
df[['experience_id','combined_id','sub_category_id']] = df[['experience_id','combined_id','sub_category_id']].astype(str)

In [106]:
df.head()

,experience_id,experience_name,city,combined_id,sub_category_id,status
0,14281.0,Super Saver Tour - Auschwitz-Birkenau & Wielic...,Krakow,2197.0,1010.0,available
2,14461.0,Tickets To Thyssen-Bornemisza Museum with Trad...,Madrid,2930.0,1002.0,available
3,15328.0,Exterior Grand Walking Tour of Prague & Prague...,Prague,1214.0,1009.0,available
4,14253.0,Conservation Drive to Al Maha Desert Resort wi...,Dubai,262.0,1073.0,available
5,14866.0,Cadaqués Walking Tour & Boat Trip,Barcelona,exp -14866.0,1010.0,available


In [107]:
df.isnull().sum()

experience_id      0
experience_name    0
city               0
combined_id        0
sub_category_id    0
status             0
dtype: int64

In [108]:
df['combo']= df['experience_name'].str.contains('Combo', regex=False).astype(int)

In [109]:
def cluster_with_cosine_way(embeddings, way = 'complete'):
  # proprietary of ethelszhang@google.com, default to consine distance
  linkage_df = sch.linkage(embeddings, method = way, metric = "cosine")
  threshold = 0.7*max(linkage_df[:,2])
  ncluster = sum([x[2] > threshold for x in linkage_df]) + 1
  # create clusters
  hc = AgglomerativeClustering(
      n_clusters = ncluster, affinity = 'cosine', linkage = way)
  # save clusters for chart
  y_hc = hc.fit_predict(embeddings)
  return y_hc

In [110]:
def similar(index,df_city):
    df_exp = list(df_city['experience_name'])
    embeddings = embed(df_exp)
    y_hc = cluster_with_cosine_way(embeddings, "complete")
    output_df = pd.DataFrame([y_hc, df_exp]).T.rename(columns = dict(zip(range(2), ["cluster", "experience_name"])))
    df_combined = df_city.join(output_df.set_index('experience_name'), on='experience_name')
    df_combined.index = df_combined['experience_name']
    df_combined = df_combined.drop(['experience_name','experience_id','status','city'],axis=1)
    df_final = pd.get_dummies(df_combined)
    rec_model.fit(df_final)
    
    df_final[df_final.index==df.iloc[index]['experience_name']]
    dist, ind = rec_model.kneighbors(df_final.iloc[index, :].values.reshape(1, -1), n_neighbors = 10)
    return list(df_final.index[ind[0]])[1:]

In [111]:
similar(1,df)

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Fast Track Tickets To Thyssen-Bornemisza National Museum',
 'Tickets To Thyssen-Bornemisza Museum with Traditional Meal',
 'Grévin Paris: Wax Museum Ticket',
 'Van Gogh Museum Tickets',
 'Carmen Thyssen Museum: Skip the Line Tickets',
 'Picasso Museum: Skip-the-line Tickets',
 'Picasso`s Birthplace Museum: Skip the Line Tickets',
 'Wax Museum Barcelona: Skip the Line Ticket',
 'Skip the Line Tickets to the Stedelijk Museum with Audioguide']

In [112]:
cities = df['city'].unique()

In [113]:
arr = []
for city in cities:
    df_city = df[df['city']==city]
    if(len(df_city) > 10):
        for i in range(0,len(df_city)):
            exp_array = similar(i,df_city)
            similar_exp = ",,".join(exp_array)
            arr.append({'experience_id':df_city.iloc[i]['experience_id'],
                        "experience_name":df_city.iloc[i]['experience_name'],
                        "similar_experience":similar_exp
                       })

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

...

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/headout1/anaconda3/lib/pytho

In [114]:
df_similar = pd.DataFrame(arr)

In [115]:
df_similar

,experience_id,experience_name,similar_experience
0,14281.0,Super Saver Tour - Auschwitz-Birkenau & Wielic...,Super Saver Tour - Auschwitz-Birkenau & Wielic...
1,14289.0,Krakow Guided Tour by E-Scooter with Food Tasting,"Krakow: Rynek Underground Museum Guided Tour,,..."
2,16090.0,Krakow: Kazimierz by Golf Cart + Schindler's F...,"Krakow: Old Town by Golf Cart, Wawel and Under..."
3,15459.0,Oskar Schindler's Story & Walking Tour,Guided Tour of Schindler's Factory and Plaszow...
4,15679.0,Panoramic Sightseeing Cruise on the Vistula Ri...,Krakow Sightseeing on a Private Gondola Cruise...
...,...,...,...
2323,13594.0,Australian Butterfly Sanctuary Entry with Guid...,Full Day Fitzroy Island Adventure Tour from Ca...
2324,13931.0,Full Day Fitzroy Island Adventure Tour from Ca...,Half Day Fitzroy Island Adventure Tour from Ca...
2325,13542.0,Cairns Aquarium by Twilight with Optional Dining,Marine Life Encounter at Cairns Aquarium With ...
2326,14498.0,Marine Life Encounter at Cairns Aquarium With ...,Marine Life Encounter at Cairns Aquarium With ...


In [116]:
df_similar.to_csv('similar.csv')